In [1]:
from ladybugtools_toolkit.box_model.box_model import BoxModel

from enum import Enum, auto
from dataclasses import dataclass, field
from honeybee.model import Room, Model 
from ladybug.epw import EPW
import json
from typing import Any, Dict, List, Optional, Union
from honeybee_energy.lib.programtypes import STANDARDS_REGISTRY
from honeybee_energy.lib.constructionsets import construction_set_by_identifier
import honeybee_energy.dictutil as energy_dict_util
from honeybee_energy.constructionset import ConstructionSet, ShadeConstruction
from honeybee_energy.construction.opaque import OpaqueConstruction
from honeybee_energy.construction.window import WindowConstruction
from honeybee_energy.material.glazing import EnergyWindowMaterialSimpleGlazSys
from honeybee_energy.programtype import ProgramType, People, ElectricEquipment, Ventilation, Lighting, Setpoint, Infiltration
from honeybee_energy.hvac import IdealAirSystem
from honeybee_energy.schedule.ruleset import ScheduleRuleset
from ladybug_geometry.geometry3d import Point3D, Face3D
from honeybee.boundarycondition import boundary_conditions
from honeybee_radiance.sensorgrid import SensorGrid
from honeybee_radiance.modifierset import ModifierSet
from honeybee_vtk.model import Model as VTKModel

In [29]:
@dataclass
class BoxGlazingParameters:
    glazing_ratio: float = field(init= True, default = 0.4)
    # targets may not be achieved, LBT will overide if needed to meet glazing ratio - TODO raise warning if not met?  
    target_window_height: float = field(init=True, default=2)
    target_sill_height: float = field(init=True, default=0.8)

    def __post_init__(self):
        if not self.glazing_ratio <= 0.95:
            raise ValueError("Glazing ratio must be less than 0.95")

@dataclass
class BoxRoomGeometry:
    glazing_parameters: BoxGlazingParameters 
    name: str = field(init=True, default='Box_Room')
    bay_width: float = field(init=True, default=3)
    count_bays: int = field(init = True, default=3)
    height: float = field(init=True, default=3)
    depth: float = field(init=True, default=10) 
    wall_thickness: float = field(init=True, default = 0.5)

    def __post_init__(self):
        # Check dimensions are within reasonable values
        if self.height < 2:
            raise ValueError("Room height must be at least 2m")
        if self.bay_width < 0.5:
            raise ValueError("Bay width must be at least 0.5m")
        if self.count_bays == 0:
            raise ValueError("There must be at least 1 bay")
        if self.depth < 2:
            raise ValueError("Room depth must be at least 2m")
        
        # Calculated values
        self.width = self.bay_width*self.count_bays # total box model width
        self.origin = Point3D(x=-self.width/2, y=-self.depth/2, z=0)

    @property
    def room(self) -> Room:
        room = Room.from_box(identifier = self.name, width = self.width, depth=self.depth, height = self.height, origin=self.origin)
        # set all faces to adiabatic
        for face in room.faces:
            face.boundary_condition = boundary_conditions.adiabatic
        # set north face (face 1) to outdoors, enables apertures to be added to this face 
        room.faces[1].boundary_condition = boundary_conditions.outdoors
        room.faces[1].apertures_by_ratio_rectangle(ratio = self.glazing_parameters.glazing_ratio,
                                                   aperture_height = self.glazing_parameters.target_window_height,
                                                   sill_height = self.glazing_parameters.target_sill_height,
                                                   horizontal_separation = self.bay_width)
        return room
    
@dataclass
class BoxSensorGrid:
    room_geometry: BoxRoomGeometry
    sensor_wall_offset: float = field(init=True, default = 0.5)
    sensor_grid_size: float = field(init=True, default = 0.2)
    sensor_grid_offset: float = field(init=True, default = 0.8)
    sensor_grid_bay_count: int = field(init=True, default = 2)

    def __post_init__(self):
        def __make_square(x, y, z, wall_thickness):
            """Returns four points that form a square with sides of length abs(x) and abs(y), 
            with a constant z coordinate."""
            return [Point3D(x, y-wall_thickness, z),
                    Point3D(-x, y-wall_thickness, z),
                    Point3D(-x, -y, z),
                    Point3D(x, -y, z)]

        vertices = __make_square(x = (self.room_geometry.bay_width/2)*self.sensor_grid_bay_count,
                                        y = (self.room_geometry.depth/2)-self.sensor_wall_offset,
                                        z = 0,
                                        wall_thickness= self.room_geometry.wall_thickness)
        face = Face3D(vertices, enforce_right_hand=False)
        mesh = face.mesh_grid(x_dim = self.sensor_grid_size, y_dim = self.sensor_grid_size, offset=self.sensor_grid_offset)
        self.sensor_grid = SensorGrid.from_planar_positions(identifier= (self.room_geometry.name + str("_sensor_grid")),
                                                            positions=mesh.face_centroids, 
                                                            plane_normal = (0,0,1)) 

@dataclass
class BoxModel:
    box_sensor_grid: BoxSensorGrid = field(init=True, default= BoxSensorGrid(room_geometry = BoxRoomGeometry(glazing_parameters=BoxGlazingParameters)))
    box_room: BoxRoomGeometry = field(init=True, default=BoxRoomGeometry(glazing_parameters=BoxGlazingParameters()))
    facade_azimuth_angle: float = field(init= True, default = 180)
    construct_set: ConstructionSet = field(init=True, default=ConstructionSet(identifier='generic_constructions'))
    modifier_set: ModifierSet = field(init=True, default=ModifierSet(identifier='generic_modifiers'))
    # TODO add other system types
    ideal_air_system: IdealAirSystem = field(init=True, default = IdealAirSystem(identifier='BM_Ideal_Air',
                                                                                 economizer_type= 'NoEconomizer',
                                                                                 demand_controlled_ventilation= False))
    program_type: ProgramType = field(init=True, default = ProgramType(identifier='generic_program_type'))

    def __post_init__(self):
        room = self.box_room.room.duplicate() # Create a copy to avoid editing input

        # Assign name
        self.name = self.box_room.name

        # Add wall thickness for daylight analysis
        if self.box_room.wall_thickness is not None and self.box_room.wall_thickness > 0:
            for aperture in room.faces[1].apertures:
                aperture.extruded_border(self.box_room.wall_thickness, True)
                room.properties.energy.construction_set = self.construct_set
        
        # Apply all properties to room
        room.properties.radiance.modifier_set = self.modifier_set
        room.properties.energy.hvac = self.ideal_air_system
        room.properties.energy.program_type = self.program_type
        model = Model(identifier = self.name, rooms = [room])
        sensor_grid = self.box_sensor_grid.sensor_grid
        model.properties.radiance.add_sensor_grids([sensor_grid])
        model.rotate_xy(angle = 360-self.facade_azimuth_angle, origin=Point3D())
        self.model = model
    
    def to_hbjson(self, name = 'BM_HBJSON', folder = None):
        return self.model.to_hbjson(name = name, folder = folder)
    
    def to_html(self, show = False, folder = None):
        # Creates a .html file of the Box Model that can be opened in browser
        if folder == None:
            vtk_model = VTKModel(self.model).to_html(name = self.name, show = show)
        else:
            vtk_model = VTKModel(self.model).to_html(name = self.name, show = show, folder = folder)
        return vtk_model

    def to_vtkjs(self):
        return VTKModel(self.model).to_vtkjs(name = self.name)
        
    def to_IES_gem(self, filepath):
        # Adds surrounding zones to be made adjacent to capture the same boundary conditions as in the HB Model version
        model = self.model.duplicate()
        top_room=Room.from_box(identifier = 'top_room', width = self.box_room.width, depth=self.box_room.depth, height = self.box_room.height, origin=self.box_room.origin.move(Vector3D(x=0, y=0, z=self.box_room.height)))
        bottom_room=Room.from_box(identifier = 'bottom_room', width = self.box_room.width, depth=self.box_room.depth, height = self.box_room.height, origin=self.box_room.origin.move(Vector3D(x=0, y=0, z=-self.box_room.height)))
        back_room=Room.from_box(identifier = 'back_room', width = self.box_room.width*3, depth=self.box_room.depth, height = self.box_room.height*3, origin=self.box_room.origin.move(Vector3D(x=-self.box_room.width, y=-self.box_room.depth, z=-self.box_room.height)))
        left_room=Room.from_box(identifier = 'left_room', width = self.box_room.width, depth=self.box_room.depth, height = self.box_room.height*3, origin=self.box_room.origin.move(Vector3D(x=-self.box_room.width, y=0, z=-self.box_room.height)))
        right_room=Room.from_box(identifier = 'right_room', width = self.box_room.width, depth=self.box_room.depth, height = self.box_room.height*3, origin=self.box_room.origin.move(Vector3D(x=self.box_room.width, y=0, z=-self.box_room.height)))
        model.add_rooms([top_room,bottom_room,back_room,left_room,right_room])
        return model.to_gem(filepath, name="BM_Gem")

    @classmethod
    def from_dict(cls, dictionary: Dict[str, Any]):
        raise NotImplementedError("create BoxModel from dict")
    
    @classmethod
    def from_json(cls, dictionary: Dict[str, Any]):
        raise NotImplementedError("create BoxModel from json")
    


In [31]:
box_glazing_params = BoxGlazingParameters(glazing_ratio=0.4,
                                          target_sill_height=0.5,
                                          target_window_height=2)

box_room_geo = BoxRoomGeometry(glazing_parameters= box_glazing_params,
                               bay_width=3,
                               count_bays=3,
                               height = 3,
                               depth = 10,
                               wall_thickness= 0.5)

box_sensor_grid = BoxSensorGrid(room_geometry= box_room_geo,
                                sensor_wall_offset= 0.5,
                                sensor_grid_offset= 0.8,
                                sensor_grid_bay_count= 2,
                                sensor_grid_size= 0.5)

box_model = BoxModel(box_sensor_grid= box_sensor_grid,
                     box_room= box_room_geo,
                     facade_azimuth_angle=180)

In [32]:
BoxRoomGeometry(glazing_parameters=BoxGlazingParameters()).room

Room: Box_Room

In [33]:
box_sensor_grid = BoxSensorGrid(room_geometry= box_room_geo,
                                sensor_wall_offset= 0.5,
                                sensor_grid_offset= 0.8,
                                sensor_grid_bay_count= 2,
                                sensor_grid_size= 0.5)

box_sensor_grid.sensor_grid

SensorGrid: Box_Room_sensor_grid [204 sensors]

In [8]:
room_geometry = BoxRoomGeometry(glazing_parameters=BoxGlazingParameters())
sensor_grid_bay_count = 2
sensor_wall_offset = 0.5

def _make_square(x, y, z, wall_thickness):
    """Returns four points that form a square with sides of length abs(x) and abs(y), 
    with a constant z coordinate."""
    return [Point3D(x, y-wall_thickness, z),
            Point3D(-x, y-wall_thickness, z),
            Point3D(-x, -y, z),
            Point3D(x, -y, z)]

vertices = _make_square(x = (room_geometry.bay_width/2)*sensor_grid_bay_count,
                                y = (room_geometry.depth/2)-sensor_wall_offset,
                                z = 0,
                                wall_thickness= room_geometry.wall_thickness)
face = Face3D(vertices, enforce_right_hand=False)

x = (room_geometry.bay_width/2)*sensor_grid_bay_count
x

3.0